In [1]:
import torch
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer, AutoModel


2022-10-21 14:53:50.795374: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-21 14:53:50.940693: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-21 14:53:50.945011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-21 14:53:50.945025: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)


In [3]:
sentence = "Bitcoin #BTC is going Up.  it is great"
labels =["Bearish", "Neutral", "Bullish"]

inputs = tokenizer.batch_encode_plus([sentence],
                                    return_tensors='pt', max_length=256,
                                    truncation=True, padding='max_length'
                                    )

In [4]:
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

input_ids.shape, attention_mask.shape

(torch.Size([1, 256]), torch.Size([1, 256]))

In [5]:
class PyTorch_to_TorchScript(torch.nn.Module):
    def __init__(self):
        super(PyTorch_to_TorchScript, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels = 3)
    def forward(self, data, attention_mask=None):
        return self.model(data, attention_mask)[0]
# after trace it will save the model in cwd
pt_model = PyTorch_to_TorchScript().eval()

# remove_attributes = []
# for key, value in vars(pt_model).items():
#     if value is None:
#         print(f'remove_attributes {key}')
#         remove_attributes.append(key)

# for key in remove_attributes:
#     delattr(pt_model, key)

traced_script_module = torch.jit.trace(pt_model, (input_ids, attention_mask), strict=False)
traced_script_module.save("./model.pt")

remove_attributes _is_full_backward_hook


/home/csh/anaconda3/envs/capstone-env/lib/python3.9/site-packages/transformers/modeling_utils.py:1967: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert all(


In [23]:
configuration = """
name: "bitcoin-model"
platform: "pytorch_libtorch"
input [
 {
    name: "input__0"
    data_type: TYPE_INT32
    dims: [ 1, 256 ]
  } ,
{
    name: "input__1"
    data_type: TYPE_INT32
    dims: [ 1, 256 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1, 3 ]
  }
"""

with open('./config.pbtxt', 'w') as file:
    file.write(configuration)